# Data Processing on Google Earth Engine

This notebook shows...
- data processing on Google Earth Engine
- exporting to GCP as TensorFlow objects

# 1. Config

In [ ]:
import ee
import geopandas as gpd
import tensorflow as tf

In [ ]:
ee.Initialize()

In [ ]:
# Required to export images
# When we export an image, we export everything within a pre-defined region
# We just set the region to be the world (the below)
# Note that we aren't sure why we have to use 88 instead of 90 - might have come from Sara!
GEOMETRY_WORLD = ee.Geometry.Polygon(
    [
        [-180, 88],
        [180, 88],
        [180, -88],
        [-180, -88],
    ],
    None,
    False
)

# 2. Data Processing

## 2.1. Fault labels

Here we will turn the vectors into a raster (image of 1s and 0s where 1 indicates a fault and 0 indicates no fault).

In [ ]:
# Read in data from EE
faultline_vectors = ee.FeatureCollection("projects/esg-satelite/assets/mars/labels/faultlines")

In [ ]:
# Add a columns of ones 
# This assigns each geometry (each fault/vector) to the value 1
# and assigns every other space to nothing (which we will fix later)
def add_column(feature):
    return feature.set({"Value": 1})

faultline_vectors_with_ones = faultline_vectors.map(add_column)

In [ ]:
faultline_im = (
    # Convert to image
    faultline_vectors_with_ones.reduceToImage(
        properties=["Value"],
        reducer=ee.Reducer.first()
    )
    # Change band name "first"  to "FAULTLINE" 
    .select(["first"], ["FAULTLINE"])
    # Unmask, which changes all non-fault pixels to have value 0
    .unmask(0)
)

In [ ]:
# Export to EE
# NOTE: There is a crs parameter that can be assigned here
# NOTE: We can play around with different scales (scale might require the crs being assigned)
task = ee.batch.Export.image.toAsset(
    faultline_im,
    description = "Faultline_image",
    assetId="projects/esg-satelite/assets/mars/labels/faultlines_raster",
    region=GEOMETRY_WORLD,
    scale=1000,
    maxPixels=1e11 
    # TODO: There is a crs parameter that can be assigned here?
)

In [ ]:
#task.start()

## 2.2. HRSC Images (sample of 3 only)

Here we will do two things
- Remove the black band from each image
- Mosaic the images together (i.e. flatten them into one image)

In [ ]:
# Read in data from EE
mars_hrsc_sample = ee.ImageCollection("projects/esg-satelite/assets/mars/mars_express_hrsc_example")

In [ ]:
# Renaming the band from "b1" to HRSC
mars_hrsc_sample = mars_hrsc_sample.select(["b1"], ["HRSC"])

In [ ]:
# Removes black band
# Applies a mask that keeps only the parts that have value greater than ("gt") 0
# Note that black is 0
def pixel_mask(image):
    return image.updateMask((image.select("HRSC").gt(0)))

mars_hrsc_filtered = mars_hrsc_sample.map(pixel_mask)

In [ ]:
# Mosaics the image collection
mars_hrsc_mosaic = mars_hrsc_filtered.mosaic()

In [ ]:
# This is the code we used to check whether the black bands were actually 0
# TODO: What does this code do again? Forgotton!
# region = mars_hrsc_sample.geometry()
# maxReducer = ee.Reducer.min()
# print(mars_hrsc_mosaic.reduceRegion(maxReducer, region, 300, bestEffort=True).getInfo())

In [ ]:
# Export to EE
# NOTE: There is a crs parameter that can be assigned here
# NOTE: We can play around with different scales (scale might require the crs being assigned)
task = ee.batch.Export.image.toAsset(
    mars_hrsc_mosaic,
    description = "HRSC_Image",
    assetId="projects/esg-satelite/assets/mars/features/post/hrsc_sample",
    region=GEOMETRY_WORLD,
    scale=1000,
    maxPixels=1e11
)

In [ ]:
# task.start()